In [5]:
import requests
import json
import time
from kafka import KafkaProducer

In [6]:
# OpenWeatherMap API Configuration
API_KEY = "5f0c7465648a9d01abf2395e97cea75c"
CITIES = ["New York", "London", "Tokyo", "Delhi", "Paris", "Berlin", "Sydney", "Toronto", "Dubai", "Singapore"]

# Kafka Producer Setup
producer = KafkaProducer(
    bootstrap_servers="localhost:9092",
    value_serializer=lambda x: json.dumps(x).encode("utf-8"),
)

In [7]:
def fetch_weather(city):
    """Fetches weather data for a given city"""
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        weather_data = response.json()
        return {
            "timestamp": weather_data["dt"],
            "city": weather_data["name"],
            "temperature": weather_data["main"]["temp"],
            "humidity": weather_data["main"]["humidity"],
            "wind_speed": weather_data["wind"]["speed"],
            "weather_condition": weather_data["weather"][0]["description"],
        }
    else:
        print(f"Error for {city}: {response.status_code} - {response.text}")
        return None


In [8]:
# Infinite loop to continuously fetch and send weather data
while True:
    for city in CITIES:
        weather_record = fetch_weather(city)
        if weather_record:
            producer.send("weather_data", value=weather_record)
            print(f"Sent: {weather_record}")
        time.sleep(2)  # Small delay to avoid API rate limits
    
    print("Waiting 30 seconds before the next round...")
    time.sleep(30)

Sent: {'timestamp': 1744199679, 'city': 'New York', 'temperature': -0.55, 'humidity': 41, 'wind_speed': 6.29, 'weather_condition': 'clear sky'}
Sent: {'timestamp': 1744199918, 'city': 'London', 'temperature': 12.65, 'humidity': 58, 'wind_speed': 4.45, 'weather_condition': 'few clouds'}
Sent: {'timestamp': 1744199719, 'city': 'Tokyo', 'temperature': 15.08, 'humidity': 49, 'wind_speed': 6.38, 'weather_condition': 'broken clouds'}
Sent: {'timestamp': 1744199674, 'city': 'Delhi', 'temperature': 40.64, 'humidity': 10, 'wind_speed': 1.25, 'weather_condition': 'clear sky'}
Sent: {'timestamp': 1744200021, 'city': 'Paris', 'temperature': 16.43, 'humidity': 35, 'wind_speed': 4.13, 'weather_condition': 'clear sky'}
Sent: {'timestamp': 1744200239, 'city': 'Berlin', 'temperature': 13.8, 'humidity': 43, 'wind_speed': 3.42, 'weather_condition': 'broken clouds'}
Sent: {'timestamp': 1744199945, 'city': 'Sydney', 'temperature': 18.53, 'humidity': 78, 'wind_speed': 4.67, 'weather_condition': 'clear sky'}

KeyboardInterrupt: 